# MLP:
The first half of this notebook may be used to train an MLP. Training for RNN models can be found in the second half of this notebook

Note that this notebook requires the use of train_df.pkl and test_df.pkl files. These are generated in the preprocess.ipynb notebook. If you have not run this notebook, you will not have the necessary data to proceed with this notebook!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.utils import get_batches, shuffle, train_val_split, preds_to_scores,scores_to_preds, plot_train_loss
from src.mlp import MLP
from src.rnn import RNN
import seaborn as sns
import plotly.plotly as py

%load_ext autoreload
%autoreload 2

In [2]:
# Define the path to the data. This is the training dataframe saved from the preprocessing notebook.
data_path = './data/train_df.pkl'
train_df = pd.read_pickle(data_path)

In [3]:
# To further isolate our data, we will only examine essays from a single set
# Feel free to experiment with different essay sets!

set = 1
df = train_df.loc[train_df['essay_set'] == set]
df.head()

,essay_id,essay_set,rater1_domain1,rater2_domain1,domain1_score,essays_embed,word_count,min_score,max_score,rater1_domain1_norm,rater2_domain1_norm,norm_score
0,1,1,4,4,8,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",299,2.0,12.0,4,4,8
1,2,1,5,4,9,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",349,2.0,12.0,5,4,9
2,3,1,4,3,7,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",236,2.0,12.0,4,3,7
4,5,1,4,4,8,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",387,2.0,12.0,4,4,8
5,6,1,4,4,8,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",204,2.0,12.0,4,4,8


In [4]:
# We should get a plot here to examine score distribution for this set
# How many essays of each score do we have?

In [5]:
# In order to avoid bias toward more common scores, we will limit the number
# of essays from each scoring bucket to a set value
score_df = None
min_score = int(df['min_score'].min())
max_score = int(df['max_score'].max())

n_max = 100
for i in range(min_score,max_score+1):
    if score_df is None:
        score_df = df.loc[df['domain1_score'] == i][:n_max]
    else:
        temp_df = df.loc[df['domain1_score'] == i][:n_max]
        score_df = pd.concat([score_df, temp_df])
df = score_df

In [6]:
# Extract essay vectors and corresponding scores
X = np.array(df['essays_embed'])
y = np.array(df['domain1_score'])
X = np.stack(X, axis=0)
print('There are {} training essays, each of shape {} x {}'.format(X.shape[0], X.shape[1], X.shape[2]))

There are 566 training essays, each of shape 426 x 200


These essays are the wrong shape to feed directly into the MLP. Therefore, each essay matrix needs to be flattened into a 1-D vector.

In [7]:
X_flatten = np.reshape(X, [X.shape[0], -1])
print('There are {} training essays, each a vector of length {}'.format(X_flatten.shape[0], X_flatten.shape[1]))

There are 566 training essays, each a vector of length 85200


The next step is to shuffle the data and separate it into training and validation sets.

In [8]:
X, y = shuffle(X_flatten, y)

X_train, y_train, X_val, y_val = train_val_split(X, y, train_prop=0.85)

Here we need to transform the labels to the form that the network will predict. For example, in set 1, the essays are graded on a scale from 2-12, therefore there are 11 classes into which the network will try to classify each essay. However, the network will classify essays into the scale 0-10. Therefore, this step will perform this shift on the labels. If the scoring range already starts at 0, no shift is performed.

In [9]:
if min_score != 0:
    y_train_adj = scores_to_preds(y_train, min_score)
    print('Training labels shifted from a scale of ({},{}) to ({},{})'\
          .format(min(y_train),max(y_train), min(y_train_adj), max(y_train_adj)))
    y_val_adj = scores_to_preds(y_val, min_score)
    print('Validation labels shifted from a scale of ({},{}) to ({},{})'\
          .format(min(y_val),max(y_val), min(y_val_adj), max(y_val_adj)))
else:
    print('No score adjustment necessary')
    y_train_adj = y_train
    y_val_adj = y_val

Training labels shifted from a scale of (2,12) to (0,10)
Validation labels shifted from a scale of (4,12) to (2,10)


### Initial MLP
Here we define an MLP model to train. The parameters below were the initial parameters tested on the dataset. This model learns the training set well, but is unable to generalize to the validation set. You may skip training this model to save time.

In [10]:
# User Defined Parameters
model_name = 'mlp_set1_bad'
hidden_dims = [128,64]
weight_scale = 1e-2
batch_size = 16
n_epochs = 20
l2_reg = 1e-4
keep_prob = 1
reg = False
lr = 1e-3

# Derived Parameters
input_dim = X_train.shape[1]
num_classes = max_score-min_score + 1
n_batches = round(X_train.shape[0]/batch_size)
batch_gen = get_batches(X_train, y_train_adj, batch_size, net_type='mlp')

mlp_net = MLP(input_dim=input_dim, hidden_dims=hidden_dims, num_classes=num_classes, weight_scale=weight_scale,\
              l2_reg=l2_reg, keep_prob=keep_prob, regression=reg)

In [11]:
print('Training Network...')
train_loss_hist, val_loss_hist = mlp_net.train(gen=batch_gen, X_val=X_val, y_val=y_val_adj, n_epochs=n_epochs, n_batches=n_batches, lr=lr,\
                                               save_every_n=5, model_name=model_name)

Training Network...


---------- Training epoch: 1 ----------
Epoch 1, Batch 1 -- Loss: 2.499 Validation accuracy: 0.202
Sample Grade Predictions: 
Preds:   6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 7 4 6 5 7
Best validation accuracy! - Saving Model


---------- Training epoch: 2 ----------
Epoch 2, Batch 1 -- Loss: 1.747 Validation accuracy: 0.202
Sample Grade Predictions: 
Preds:   8 8 8 8 6 8 8 8 8 6 8 6 8 8 8 8 6 8 8 8
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 7 4 6 5 7


---------- Training epoch: 3 ----------
Epoch 3, Batch 1 -- Loss: 1.467 Validation accuracy: 0.357
Sample Grade Predictions: 
Preds:   7 8 6 8 4 8 8 8 8 4 4 4 7 8 7 7 4 7 4 7
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 7 4 6 5 7
Best validation accuracy! - Saving Model


---------- Training epoch: 4 ----------
Epoch 4, Batch 1 -- Loss: 0.654 Validation accuracy: 0.476
Sample Grade Predictions: 
Preds:   6 8 5 8 4 8 8 8 7 4 5 4 6 6 6 6 4 6 5 6
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 

In [12]:
fig = plot_train_loss(train_loss_hist, val_loss_hist, n_batches, model_name)
py.iplot(fig, filename='basic-area')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ac4391/0 or inside your plot.ly account where it is named 'basic-area'


/home/ecbm4040/miniconda3/envs/EssayGrader_env/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### Train your own MLP
Clearly the MLP above is able to learn the training set, but is unable to generalize for the validation set. Below is another MLP model definition. The user may change the model name and parameters, or leave the model definition as is. The model will be saved to the 'model/' directory of this project. Parameters such as the following may be defined by the user: learning rate, number of training epochs, l2 regularization, dropout probability, and regression vs classification.

After many iterations, we found the following mlp parameters yielded the best results on both the training and validation sets. Note that this model is much larger and requires a GPU to train in a reasonable amount of time. 

In [13]:
# User Defined Parameters
model_name = 'mlp_set'+'{}'.format(set)
hidden_dims = [1024,256]
weight_scale = 1e-2
batch_size = 16
n_epochs = 20
l2_reg = 1e-4
keep_prob = 0.6
reg = False
lr = 1e-4

# Derived Parameters
input_dim = X_train.shape[1]
num_classes = max_score-min_score + 1
n_batches = round(X_train.shape[0]/batch_size)
batch_gen = get_batches(X_train, y_train_adj, batch_size, net_type='mlp')

mlp_net = MLP(input_dim=input_dim, hidden_dims=hidden_dims, num_classes=num_classes, weight_scale=weight_scale,\
              l2_reg=l2_reg, keep_prob=keep_prob, regression=reg)

In [14]:
print('Training Network...')
train_loss_hist, val_loss_hist = mlp_net.train(gen=batch_gen, X_val=X_val, y_val=y_val_adj, n_epochs=n_epochs, n_batches=n_batches, lr=lr,\
                                               save_every_n=5, model_name=model_name)

Training Network...


---------- Training epoch: 1 ----------
Epoch 1, Batch 1 -- Loss: 6.132 Validation accuracy: 0.143
Sample Grade Predictions: 
Preds:   6 7 6 6 2 8 7 6 6 6 2 2 6 8 8 6 6 6 2 8
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 7 4 6 5 7
Best validation accuracy! - Saving Model


---------- Training epoch: 2 ----------
Epoch 2, Batch 1 -- Loss: 1.996 Validation accuracy: 0.143
Sample Grade Predictions: 
Preds:   7 8 7 9 7 9 7 7 8 9 9 7 8 8 8 5 6 7 7 9
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 7 4 6 5 7


---------- Training epoch: 3 ----------
Epoch 3, Batch 1 -- Loss: 1.998 Validation accuracy: 0.214
Sample Grade Predictions: 
Preds:   7 7 7 7 7 7 7 7 5 4 5 5 8 7 8 7 8 7 7 7
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 7 4 6 5 7
Best validation accuracy! - Saving Model


---------- Training epoch: 4 ----------
Epoch 4, Batch 1 -- Loss: 1.892 Validation accuracy: 0.357
Sample Grade Predictions: 
Preds:   7 7 6 8 4 8 8 8 8 5 5 5 8 8 8 7 5 8 4 7
Actual:  6 8 5 8 2 7 6 10 7 4 6 4 7 7 5 

In [15]:
fig = plot_train_loss(train_loss_hist, val_loss_hist, n_batches, model_name)
py.iplot(fig, filename='basic-area')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ac4391/0 or inside your plot.ly account where it is named 'basic-area'


## Test the QWK of the trained model
Now we can use essays from the test dataset to obtain a quadratic weighted
kappa (QWK) score for the model. This metric is used to quantify how well
the model predicted the essay scores relative to random guessing. A value
of 0 indicates that the predictions were no better than random guessing,
while a value of 1 indicates perfect matching between predictions and labels.

In [16]:
data_path = './data/test_df.pkl'
test_df = pd.read_pickle(data_path)
df = test_df.loc[test_df['essay_set'] == set]
X_test = np.array(df['essays_embed'])
y_test = np.array(df['domain1_score'])
X_test = np.stack(X_test, axis=0)
X_test = np.reshape(X_test, [X_test.shape[0], -1])
print('There are {} testing essays'.format(X_test.shape[0]))
      
if min_score != 0:
    y_test_adj = scores_to_preds(y_test, min_score)
    print('Testing labels shifted from a scale of ({},{}) to ({},{})'\
          .format(min(y_test),max(y_test), min(y_test_adj), max(y_test_adj)))
else:
    print('No score adjustment necessary')
    y_test_adj = y_test

There are 298 testing essays
Testing labels shifted from a scale of (4,12) to (2,10)


In [17]:
preds = mlp_net.predict('./model/'+model_name, X_test)

# We need to map predictions from classes in the model to actual scores
#preds = preds_to_scores(preds, min_score=min_score)

INFO:tensorflow:Restoring parameters from ./model/mlp_set1


In [18]:
from src.utils import quadratic_weighted_kappa
y_test_adj = scores_to_preds(y_test, min_score)
k = quadratic_weighted_kappa(y_test_adj, preds, num_classes)

print('The quadratic weighted kappa score for set {} using {} is : {}'\
     .format(set, model_name, k))

The quadratic weighted kappa score for set 1 using mlp_set1 is : 0.7055854051347291


# RNN:
The second half of this notebook may be used for training an RNN - specifically an LSTM or GRU

In [27]:
# Define the path to the data
data_path = './data/train_df.pkl'
train_df = pd.read_pickle(data_path)

# To further isolate our data, we will only examine essays from a single set
# Feel free to experiment with different essay sets!
set = 1
df = train_df.loc[train_df['essay_set'] == set]
df.head()

,essay_id,essay_set,rater1_domain1,rater2_domain1,domain1_score,essays_embed,word_count,min_score,max_score,rater1_domain1_norm,rater2_domain1_norm,norm_score
0,1,1,4,4,8,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",299,2.0,12.0,4,4,8
1,2,1,5,4,9,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",349,2.0,12.0,5,4,9
2,3,1,4,3,7,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",236,2.0,12.0,4,3,7
4,5,1,4,4,8,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",387,2.0,12.0,4,4,8
5,6,1,4,4,8,"[[0.1285, 0.68849, 0.83504, -0.16483, -0.36831...",204,2.0,12.0,4,4,8


In [28]:
# In order to avoid bias toward more common scores, we will limit the number
# of essays from each scoring bucket to a set value
score_df = None
min_score = int(df['min_score'].min())
max_score = int(df['max_score'].max())

n_max = 100
for i in range(min_score,max_score+1):
    if score_df is None:
        score_df = df.loc[df['domain1_score'] == i][:n_max]
    else:
        temp_df = df.loc[df['domain1_score'] == i][:n_max]
        score_df = pd.concat([score_df, temp_df])
df = score_df

In [29]:
# Extract essay vectors and corresponding scores
X = np.array(df['essays_embed'])
y = np.array(df['domain1_score'])
X = np.stack(X, axis=0)
print('There are {} training essays, each of shape {} x {}'.format(X.shape[0], X.shape[1], X.shape[2]))

There are 566 training essays, each of shape 426 x 200


The next step is to shuffle the data and separate it into training and validation sets.

In [30]:
X, y = shuffle(X, y)
X_train, y_train, X_val, y_val = train_val_split(X, y, train_prop=0.85)

Here we need to transform the labels to the form that the network will predict. For example, in set 1, the essays are graded on a scale from 2-12, therefore there are 11 classes into which the network will try to classify each essay. However, the network will classify essays into the scale 0-10. Therefore, this step will perform this shift on the labels.

In [31]:
if min_score != 0:
    y_train_adj = scores_to_preds(y_train, min_score)
    print('Training labels shifted from a scale of ({},{}) to ({},{})'\
          .format(min(y_train),max(y_train), min(y_train_adj), max(y_train_adj)))
    y_val_adj = scores_to_preds(y_val, min_score)
    print('Validation labels shifted from a scale of ({},{}) to ({},{})'\
          .format(min(y_val),max(y_val), min(y_val_adj), max(y_val_adj)))
else:
    print('No score adjustment necessary')
    y_train_adj = y_train
    y_val_adj = y_val

Training labels shifted from a scale of (2,12) to (0,10)
Validation labels shifted from a scale of (2,12) to (0,10)


### Initial RNN
Here we define an RNN model to train. The parameters below were the initial parameters tested on the dataset. model learns the training and validation set well. It serves as a good baseline from which you can design your own RNN. If you'd like, you may skip training this model to save time and move directly to training your own model with tunable parameters.

In [32]:
# User Defined Parameters
batch_size = 32
cell_type = 'lstm'
rnn_size = 128
lr = 1e-3
n_epochs = 20
keep_prob = 1

# Derived Parameters
model_name = cell_type+'_set'+'{}'.format(set)
num_classes = max_score-min_score + 1
n_batches = round(X_train.shape[0]/batch_size)
seq_length = X_train.shape[1]
embed_size = X_train.shape[2]

X_val_t = X_val[:batch_size]
y_val_t = y_val_adj[:batch_size]
batch_gen = get_batches(X_train, y_train_adj, batch_size, net_type=cell_type)

rnn_net = RNN(num_classes, batch_size, seq_length, embed_size, cell_type=cell_type,
                 rnn_size=rnn_size, learning_rate=lr, train_keep_prob=1)

In [33]:
print('Training Network...')
train_loss_hist, val_loss_hist = rnn_net.train(batch_gen, X_val_t, y_val_t,\
                                              n_epochs, n_batches, save_every_n=5,\
                                              model_name=model_name)

Training Network...


---------- Training epoch: 1 ----------
Epoch 1, step 5 loss: 2.3688  validation accuracy: 0.40625  0.4050 sec/batch
Best validation accuracy! - Saving Model
Sample Grade Predictions: 
Preds:   5 5 5 5 5 8 5 5 5 8 5 8 5 5 5 5 5 5 8 5
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 1, step 10 loss: 2.3755  validation accuracy: 0.1875  0.4160 sec/batch
Sample Grade Predictions: 
Preds:   8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 1, step 15 loss: 2.1035  validation accuracy: 0.40625  0.4128 sec/batch
Sample Grade Predictions: 
Preds:   5 5 5 5 5 8 5 5 5 8 5 8 5 5 5 5 5 5 8 5
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8


---------- Training epoch: 2 ----------
Epoch 2, step 5 loss: 2.2533  validation accuracy: 0.40625  0.4122 sec/batch
Sample Grade Predictions: 
Preds:   5 5 5 5 5 8 5 5 5 8 5 8 5 5 5 5 5 5 8 5
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 2, step 10 loss: 1.9323  validation accu

Epoch 13, step 15 loss: 1.2707  validation accuracy: 0.53125  0.4043 sec/batch
Best validation accuracy! - Saving Model
Sample Grade Predictions: 
Preds:   6 7 5 5 5 8 4 7 4 8 6 8 5 6 5 5 7 5 8 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8


---------- Training epoch: 14 ----------
Epoch 14, step 5 loss: 1.5246  validation accuracy: 0.46875  0.4276 sec/batch
Sample Grade Predictions: 
Preds:   6 8 5 5 5 8 4 8 4 8 6 8 5 6 5 5 7 5 8 8
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 14, step 10 loss: 1.5516  validation accuracy: 0.375  0.4022 sec/batch
Sample Grade Predictions: 
Preds:   6 8 5 5 5 8 4 8 4 8 6 8 5 6 6 5 8 6 8 8
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 14, step 15 loss: 1.5533  validation accuracy: 0.5  0.3929 sec/batch
Sample Grade Predictions: 
Preds:   6 7 5 6 5 8 4 7 4 8 6 8 5 6 6 5 7 6 8 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8


---------- Training epoch: 15 ----------
Epoch 15, step 5 loss: 1.2843  validation accuracy: 0.46875  0.4

In [35]:
fig = plot_train_loss(train_loss_hist, val_loss_hist, n_batches, model_name)
py.iplot(fig, filename='basic-area')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ac4391/0 or inside your plot.ly account where it is named 'basic-area'


### Train your own RNN
The LSTM above is able to learn the training set and performance on the validation set is comparable. These preliminary results are promising, but changing hyperparameters can yield even better results. Below is another RNN model definition. Again, many parameters can be modified by the user or left alone with the parameters that yielded our best results.The model will be saved to the 'model/' directory of this project. 

After many iterations, we found the following mlp parameters yielded the best results on both the training and validation sets:

In [36]:
# User Defined Parameters

batch_size = 32
cell_type = 'gru'
rnn_size = 256
lr = 1e-3
n_epochs = 20
keep_prob = 1

# Derived Parameters
model_name = cell_type+'_set'+'{}'.format(set)
num_classes = max_score-min_score + 1
n_batches = round(X_train.shape[0]/batch_size)
seq_length = X_train.shape[1]
embed_size = X_train.shape[2]

X_val_t = X_val[:batch_size]
y_val_t = y_val_adj[:batch_size]
batch_gen = get_batches(X_train, y_train_adj, batch_size, net_type=cell_type)

rnn_net = RNN(num_classes, batch_size, seq_length, embed_size, cell_type=cell_type,
                 rnn_size=rnn_size, learning_rate=lr, train_keep_prob=1)

In [37]:
print('Training Network...')
train_loss_hist, val_loss_hist = rnn_net.train(batch_gen, X_val_t, y_val_t,\
                                              n_epochs, n_batches, save_every_n=2,\
                                              model_name=model_name)

Training Network...


---------- Training epoch: 1 ----------
Epoch 1, step 5 loss: 2.2509  validation accuracy: 0.1875  0.4658 sec/batch
Best validation accuracy! - Saving Model
Sample Grade Predictions: 
Preds:   8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 1, step 10 loss: 2.1954  validation accuracy: 0.375  0.4673 sec/batch
Best validation accuracy! - Saving Model
Sample Grade Predictions: 
Preds:   4 7 4 4 4 7 4 7 4 7 4 7 4 4 4 4 7 4 7 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 1, step 15 loss: 2.2256  validation accuracy: 0.375  0.4617 sec/batch
Sample Grade Predictions: 
Preds:   4 7 4 4 4 7 4 7 4 8 4 7 4 4 4 4 7 4 8 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8


---------- Training epoch: 2 ----------
Epoch 2, step 5 loss: 1.8898  validation accuracy: 0.1875  0.4495 sec/batch
Sample Grade Predictions: 
Preds:   8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 2, s

Epoch 13, step 10 loss: 1.3675  validation accuracy: 0.53125  0.5126 sec/batch
Sample Grade Predictions: 
Preds:   6 7 5 5 5 8 4 7 3 8 6 8 5 6 6 5 7 6 8 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 13, step 15 loss: 1.2480  validation accuracy: 0.40625  0.4711 sec/batch
Sample Grade Predictions: 
Preds:   6 7 5 5 5 8 4 8 3 8 6 8 5 6 6 5 6 6 8 8
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8


---------- Training epoch: 14 ----------
Epoch 14, step 5 loss: 1.1263  validation accuracy: 0.3125  0.4602 sec/batch
Sample Grade Predictions: 
Preds:   6 6 5 5 4 8 4 6 0 8 6 8 5 6 6 5 6 6 8 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 14, step 10 loss: 1.2460  validation accuracy: 0.5  0.4528 sec/batch
Sample Grade Predictions: 
Preds:   6 7 5 5 5 7 4 7 4 8 6 7 5 6 6 5 7 6 8 7
Actual:  7 7 5 4 5 8 4 7 2 9 8 7 5 5 5 4 7 5 7 8
Epoch 14, step 15 loss: 1.2397  validation accuracy: 0.59375  0.4671 sec/batch
Sample Grade Predictions: 
Preds:   6 7 5 5 5 7 4 7 2 8 6 8 5 6 5 5 

In [38]:
fig = plot_train_loss(train_loss_hist, val_loss_hist, n_batches, model_name)
py.iplot(fig, filename='basic-area')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ac4391/0 or inside your plot.ly account where it is named 'basic-area'


## Test the QWK of the trained model
Now we can use essays from the test dataset to obtain a quadratic weighted
kappa (QWK) score for the model. This metric is used to quantify how well
the model predicted the essay scores relative to random guessing. A value
of 0 indicates that the predictions were no better than random guessing,
while a value of 1 indicates perfect matching between predictions and labels.

In [39]:
data_path = './data/test_df.pkl'
test_df = pd.read_pickle(data_path)
df = test_df.loc[test_df['essay_set'] == set]
X_test = np.array(df['essays_embed'])
y_test = np.array(df['domain1_score'])
X_test = np.stack(X_test, axis=0)

print('There are {} testing essays'.format(X_test.shape[0]))
      
if min_score != 0:
    y_test_adj = scores_to_preds(y_test, min_score)
    print('Testing labels shifted from a scale of ({},{}) to ({},{})'\
          .format(min(y_test),max(y_test), min(y_test_adj), max(y_test_adj)))
else:
    print('No score adjustment necessary')
    y_test_adj = y_test

There are 298 testing essays
Testing labels shifted from a scale of (4,12) to (2,10)


In [40]:
batch_size = X_test.shape[0]
seq_length = X_test.shape[1]
embed_size = X_test.shape[2]

pred_net = RNN(num_classes, batch_size, seq_length, embed_size, cell_type=cell_type,
                 rnn_size=rnn_size, learning_rate=lr, train_keep_prob=1)
preds = pred_net.predict('./model/'+model_name, X_test)


INFO:tensorflow:Restoring parameters from ./model/gru_set1
Running network predictions


In [41]:
k = quadratic_weighted_kappa(preds[0], y_test_adj, num_classes)

print('The quadratic weighted kappa score for set {} using {} is : {}'\
     .format(set, model_name, k))

The quadratic weighted kappa score for set 1 using gru_set1 is : 0.6958062221753905
